# DPG for Cartpole


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

/home/aras/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [6]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.5148954174114357 -2.870291843392152
actions: 1 0
rewards: 1.0 1.0


In [7]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [8]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [10]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    rates = tf.placeholder(tf.float32, [None], name='rates')
    return states, actions, targetQs, rates

In [11]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [12]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [13]:
def model_loss(action_size, hidden_size, states, actions, targetQs, rates):
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                              labels=actions_labels)
    targetQs = tf.reshape(targetQs, shape=[-1, 1])
    gloss = tf.reduce_mean(neg_log_prob * targetQs) # DPG: r+(gamma*nextQ)
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, reuse=True) # Qs
    rates = tf.reshape(rates, shape=[-1, 1])
    dlossA = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=rates)) # 0-1
    dlossA += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                     labels=rates)) # 0-1
    dlossA /= 2
    dlossQ = tf.reduce_mean(tf.square(gQs - targetQs)) # DQN
    dlossQ += tf.reduce_mean(tf.square(dQs - targetQs)) # DQN
    dlossQ /= 2
    return actions_logits, gQs, gloss, dlossA, dlossQ

In [14]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_lossA, d_lossQ, g_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss, var_list=g_vars)
        d_optA = tf.train.AdamOptimizer(d_learning_rate).minimize(d_lossA, var_list=d_vars)
        d_optQ = tf.train.AdamOptimizer(d_learning_rate).minimize(d_lossQ, var_list=d_vars)

    return g_opt, d_optA, d_optQ

In [15]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, g_learning_rate, d_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_lossA, self.d_lossQ = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_optA, self.d_optQ = model_opt(g_loss=self.g_loss, 
                                                         d_lossA=self.d_lossA, 
                                                         d_lossQ=self.d_lossQ, 
                                                         g_learning_rate=g_learning_rate, 
                                                         d_learning_rate=d_learning_rate)

In [16]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
        self.rates = deque(maxlen=max_size) # rates
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), # ==  self.rates
#                                size=batch_size, 
#                                replace=False)
#         return [self.buffer[ii] for ii in idx], [self.rates[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [17]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [18]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
g_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [19]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              g_learning_rate=g_learning_rate, d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [20]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    memory.rates.append(-1) # empty
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        total_reward = 0 # reset
        for idx in range(num_step): # episode length
            if memory.rates[-1-idx] == -1:
                memory.rates[-1-idx] = rate
        num_step = 0 # reset

In [32]:
# batch = memory.buffer
# percentage = 0.9
# for idx in range(memory_size// batch_size):
#     #batch, rates_ = memory.sample(batch_size=batch_size)
#     rates = np.array(memory.rates)[idx*batch_size:(idx+1)*batch_size]
#     states = np.array([each[0] for each in batch])[rates >= (np.max(rates)*percentage)]
#     actions = np.array([each[1] for each in batch])[rates >= (np.max(rates)*percentage)]
#     next_states = np.array([each[2] for each in batch])[rates >= (np.max(rates)*percentage)]
#     rewards = np.array([each[3] for each in batch])[rates >= (np.max(rates)*percentage)]
#     dones = np.array([each[4] for each in batch])[rates >= (np.max(rates)*percentage)]
#     maxrates = rates[rates >= (np.max(rates)*percentage)]
#     print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, maxrates.shape)

In [33]:
# batch = memory.buffer
# percentage = 0.9
# for idx in range(memory_size// batch_size):
#     states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
#     actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
#     next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
#     rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
#     dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
#     rates = np.array(memory.rates)[idx*batch_size:(idx+1)*batch_size]
#     print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape)
#     states = states[rates >= 0]
#     actions = actions[rates >= 0]
#     next_states = next_states[rates >= 0]
#     rewards = rewards[rates >= 0]
#     dones = dones[rates >= 0]
#     rates = rates[rates >= 0]
#     print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape)

In [34]:
batch = memory.buffer
percentage = 0.9
# statesL, actionsL, next_statesL, rewardsL, donesL, ratesL = [], [], [], [], [], []
for idx in range(memory_size// batch_size):
    states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
    actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
    next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
    rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
    dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
    rates = np.array(memory.rates)[idx*batch_size:(idx+1)*batch_size]
    print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape)
    states = states[rates >= (np.max(rates)*percentage)]
    actions = actions[rates >= (np.max(rates)*percentage)]
    next_states = next_states[rates >= (np.max(rates)*percentage)]
    rewards = rewards[rates >= (np.max(rates)*percentage)]
    dones = dones[rates >= (np.max(rates)*percentage)]
    rates = rates[rates >= (np.max(rates)*percentage)]
    print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape)
    # statesL.append(states)
    # actionsL.append(actions)
    # next_statesL.append(next_states)
    # rewardsL.append(rewards)
    # donesL.append(dones)
    # ratesL.append(rates)

(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(193, 4) (193,) (193, 4) (193,) (193,) (193,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(346, 4) (346,) (346, 4) (346,) (346,) (346,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(89, 4) (89,) (89, 4) (89,) (89,) (89,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(156, 4) (156,) (156, 4) (156,) (156,) (156,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(91, 4) (91,) (91, 4) (91,) (91,) (91,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(142, 4) (142,) (142, 4) (142,) (142,) (142,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(99, 4) (99,) (99, 4) (99,) (99,) (99,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(167, 4) (167,) (167, 4) (167,) (167,) (167,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(107, 4) (107,) (107, 4) (107,) (107,) (107,)
(10000, 4) (10000,) (10000, 4) (10000,) (10000,) (10000,)
(72, 4) (72,) (72, 4) (7

In [35]:
states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape

((72, 4), (72,), (72, 4), (72,), (72,), (72,))

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list = [] # goal
rewards_list, gloss_list, dloss_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(1111):
        total_reward = 0 # each episode
        gloss_batch, dlossA_batch, dlossQ_batch= [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        idx_arr = np.arange(memory_size// batch_size)

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
                #print(action)
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits) # adding epsilon*noise
                #print(action)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.rates.append(-1) # empty
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.rates[-1-idx] == -1: # double-check the landmark/marked indexes
                        memory.rates[-1-idx] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            percentage = 0.9
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(idx_arr)
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array(memory.rates)[idx*batch_size:(idx+1)*batch_size]
            #print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape)
            states = states[rates >= (np.max(rates)*percentage)]
            actions = actions[rates >= (np.max(rates)*percentage)]
            next_states = next_states[rates >= (np.max(rates)*percentage)]
            rewards = rewards[rates >= (np.max(rates)*percentage)]
            dones = dones[rates >= (np.max(rates)*percentage)]
            rates = rates[rates >= (np.max(rates)*percentage)]
            #print(states.shape, actions.shape, next_states.shape, rewards.shape, dones.shape, rates.shape)
            nextQs_logits = sess.run(model.Qs_logits, feed_dict = {model.states: next_states})
            #nextQs = np.max(nextQs_logits, axis=1) * (1-dones) # DQN
            nextQs = nextQs_logits.reshape([-1]) * (1-dones) # DPG
            targetQs = rewards + (gamma * nextQs)
            gloss, dlossA, dlossQ, _, _, _ = sess.run([model.g_loss, model.d_lossA, model.d_lossQ, 
                                                       model.g_opt, model.d_optA, model.d_optQ],
                                                      feed_dict = {model.states: states, 
                                                                   model.actions: actions,
                                                                   model.targetQs: targetQs, 
                                                                   model.rates: rates})
            gloss_batch.append(gloss)
            dlossA_batch.append(dlossA)
            dlossQ_batch.append(dlossQ)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'gloss:{:.4f}'.format(np.mean(gloss_batch)),
              'dlossA:{:.4f}'.format(np.mean(dlossA_batch)),
              'dlossQ:{:.4f}'.format(np.mean(dlossQ_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        #gloss_list.append([ep, np.mean(gloss_batch)])
        #dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:12.0000 R:12.0 rate:0.024 gloss:0.6205 dlossA:0.6656 dlossQ:1.1547 exploreP:0.9988
Episode:1 meanR:18.5000 R:25.0 rate:0.05 gloss:0.6211 dlossA:0.6628 dlossQ:1.1682 exploreP:0.9963
Episode:2 meanR:21.0000 R:26.0 rate:0.052 gloss:0.6172 dlossA:0.6609 dlossQ:1.1629 exploreP:0.9938
Episode:3 meanR:20.0000 R:17.0 rate:0.034 gloss:0.6149 dlossA:0.6591 dlossQ:1.1679 exploreP:0.9921
Episode:4 meanR:18.0000 R:10.0 rate:0.02 gloss:0.6140 dlossA:0.6640 dlossQ:1.1479 exploreP:0.9911
Episode:5 meanR:21.8333 R:41.0 rate:0.082 gloss:0.6167 dlossA:0.6620 dlossQ:1.1639 exploreP:0.9871
Episode:6 meanR:20.7143 R:14.0 rate:0.028 gloss:0.6138 dlossA:0.6607 dlossQ:1.1630 exploreP:0.9857
Episode:7 meanR:19.7500 R:13.0 rate:0.026 gloss:0.6163 dlossA:0.6606 dlossQ:1.1733 exploreP:0.9845
Episode:8 meanR:19.7778 R:20.0 rate:0.04 gloss:0.6167 dlossA:0.6601 dlossQ:1.1772 exploreP:0.9825
Episode:9 meanR:20.0000 R:22.0 rate:0.044 gloss:0.6185 dlossA:0.6601 dlossQ:1.1817 exploreP:0.9804
Episode:10 me

Episode:83 meanR:30.3095 R:55.0 rate:0.11 gloss:0.6023 dlossA:0.6675 dlossQ:1.1330 exploreP:0.7775
Episode:84 meanR:30.1529 R:17.0 rate:0.034 gloss:0.6012 dlossA:0.6681 dlossQ:1.1254 exploreP:0.7762
Episode:85 meanR:30.0233 R:19.0 rate:0.038 gloss:0.6072 dlossA:0.6696 dlossQ:1.1331 exploreP:0.7747
Episode:86 meanR:30.3333 R:57.0 rate:0.114 gloss:0.6009 dlossA:0.6658 dlossQ:1.1301 exploreP:0.7704
Episode:87 meanR:30.3523 R:32.0 rate:0.064 gloss:0.6036 dlossA:0.6658 dlossQ:1.1353 exploreP:0.7679
Episode:88 meanR:30.2472 R:21.0 rate:0.042 gloss:0.5946 dlossA:0.6639 dlossQ:1.1455 exploreP:0.7664
Episode:89 meanR:31.3222 R:127.0 rate:0.254 gloss:0.5986 dlossA:0.6649 dlossQ:1.1351 exploreP:0.7568
Episode:90 meanR:32.0549 R:98.0 rate:0.196 gloss:0.6007 dlossA:0.6655 dlossQ:1.1379 exploreP:0.7495
Episode:91 meanR:31.8043 R:9.0 rate:0.018 gloss:0.5885 dlossA:0.6603 dlossQ:1.1481 exploreP:0.7489
Episode:92 meanR:31.6989 R:22.0 rate:0.044 gloss:0.6008 dlossA:0.6667 dlossQ:1.1352 exploreP:0.7472
E

Episode:165 meanR:70.1400 R:244.0 rate:0.488 gloss:0.6950 dlossA:0.7252 dlossQ:1.0001 exploreP:0.4143
Episode:166 meanR:71.5100 R:159.0 rate:0.318 gloss:0.7054 dlossA:0.7256 dlossQ:0.9923 exploreP:0.4079
Episode:167 meanR:74.8800 R:361.0 rate:0.722 gloss:0.7167 dlossA:0.7350 dlossQ:0.9822 exploreP:0.3938
Episode:168 meanR:76.9600 R:222.0 rate:0.444 gloss:0.7267 dlossA:0.7435 dlossQ:0.9803 exploreP:0.3854
Episode:169 meanR:79.7600 R:322.0 rate:0.644 gloss:0.7309 dlossA:0.7456 dlossQ:0.9890 exploreP:0.3735
Episode:170 meanR:80.5600 R:131.0 rate:0.262 gloss:0.7356 dlossA:0.7420 dlossQ:0.9841 exploreP:0.3688
Episode:171 meanR:82.8000 R:239.0 rate:0.478 gloss:0.7454 dlossA:0.7540 dlossQ:0.9696 exploreP:0.3603
Episode:172 meanR:86.6000 R:408.0 rate:0.816 gloss:0.7547 dlossA:0.7618 dlossQ:0.9618 exploreP:0.3463
Episode:173 meanR:90.3100 R:419.0 rate:0.838 gloss:0.7645 dlossA:0.7687 dlossQ:0.9558 exploreP:0.3325
Episode:174 meanR:92.6900 R:255.0 rate:0.51 gloss:0.7721 dlossA:0.7640 dlossQ:0.95

Episode:246 meanR:347.2900 R:500.0 rate:1.0 gloss:5.8539 dlossA:4.7761 dlossQ:1.5203 exploreP:0.0256
Episode:247 meanR:348.3700 R:303.0 rate:0.606 gloss:6.0015 dlossA:4.7427 dlossQ:1.4859 exploreP:0.0251
Episode:248 meanR:352.9400 R:500.0 rate:1.0 gloss:6.1646 dlossA:4.8433 dlossQ:1.5560 exploreP:0.0244
Episode:249 meanR:356.4200 R:500.0 rate:1.0 gloss:6.6527 dlossA:4.9133 dlossQ:1.6127 exploreP:0.0237
Episode:250 meanR:360.2400 R:500.0 rate:1.0 gloss:7.1341 dlossA:5.3243 dlossQ:1.8980 exploreP:0.0230
Episode:251 meanR:364.8900 R:500.0 rate:1.0 gloss:7.3818 dlossA:5.9522 dlossQ:2.0194 exploreP:0.0224
Episode:252 meanR:368.5900 R:500.0 rate:1.0 gloss:7.7569 dlossA:5.8168 dlossQ:2.3246 exploreP:0.0218
Episode:253 meanR:373.4700 R:500.0 rate:1.0 gloss:7.8622 dlossA:6.2360 dlossQ:2.3337 exploreP:0.0212
Episode:254 meanR:377.7100 R:500.0 rate:1.0 gloss:8.3947 dlossA:5.9505 dlossQ:2.3900 exploreP:0.0207
Episode:255 meanR:381.2500 R:500.0 rate:1.0 gloss:8.9207 dlossA:6.6697 dlossQ:2.9710 expl

Episode:326 meanR:477.1400 R:500.0 rate:1.0 gloss:21.2632 dlossA:8.3736 dlossQ:9.6350 exploreP:0.0103
Episode:327 meanR:477.1400 R:500.0 rate:1.0 gloss:21.7871 dlossA:10.1387 dlossQ:10.4156 exploreP:0.0103
Episode:328 meanR:480.6700 R:500.0 rate:1.0 gloss:21.5989 dlossA:10.3015 dlossQ:10.6448 exploreP:0.0103
Episode:329 meanR:480.6700 R:500.0 rate:1.0 gloss:21.4148 dlossA:11.2631 dlossQ:11.1940 exploreP:0.0103
Episode:330 meanR:480.6700 R:500.0 rate:1.0 gloss:21.2416 dlossA:9.4805 dlossQ:8.9223 exploreP:0.0103
Episode:331 meanR:480.6700 R:500.0 rate:1.0 gloss:21.4556 dlossA:9.0298 dlossQ:9.4877 exploreP:0.0103


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [36]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.